In [1]:
import numpy as np
import matplotlib.pyplot as plt
from numpy import unravel_index

from skimage import data
from skimage.registration import phase_cross_correlation
from skimage.registration._phase_cross_correlation import _upsampled_dft
from scipy.ndimage import fourier_shift

import warnings

In [2]:
%pylab qt

Populating the interactive namespace from numpy and matplotlib


In [3]:
import hyperspy.api as hs

In [16]:
path = "/Users/josephvas/Dropbox/check_figures/"
file_out = "holo_state_00OL_value 25.npy"
file_in = "holo_state_01OL_value 25.npy"

TEM2 = hs.signals.Signal2D(np.load(path+file_out))
TEM3 = hs.signals.Signal2D(np.load(path+file_in))

#TEM1 = np.load(path+file_out) #underfocus
#TEM2 = np.load(path+file_in) #in-focus

In [17]:
def holograph_reconstruction(TEM1):
    TEM1.set_signal_type('hologram')
    sideband_pos = TEM1.estimate_sideband_position(ap_cb_radius=None, sb='lower')
    sideband_size = TEM1.estimate_sideband_size(sideband_pos)
    wave_image = TEM1.reconstruct_phase(sb_size = sideband_size, sb_position =sideband_pos)
    wave_imge2 = wave_image.unwrapped_phase()
    
    return(np.abs(wave_image),wave_imge2)

In [18]:
(absolute1, phase1) = holograph_reconstruction(TEM2)
(absolute2, phase2) = holograph_reconstruction(TEM3)
warnings.filterwarnings(action='once')

In [9]:
# subpixel precision
shift, error, diffphase = phase_cross_correlation(absolute1.data, absolute2.data,
                                                  upsample_factor=100)

fig = plt.figure(figsize=(8, 3))
ax1 = plt.subplot(1, 3, 1)
ax2 = plt.subplot(1, 3, 2, sharex=ax1, sharey=ax1)
ax3 = plt.subplot(1, 3, 3)

ax1.imshow(absolute1.data, cmap='gray')
#ax1.set_axis_off()
ax1.set_title('Reference image')

ax2.imshow(absolute2.data, cmap='gray')
#ax2.set_axis_off()
ax2.set_title('Offset image')

ax3.imshow(absolute2.data-absolute1.data, cmap='gray')
ax3.set_title("Cross-correlation")

plt.show()

print(f'Detected subpixel offset (y, x): {shift}')

Detected subpixel offset (y, x): [0. 0.]


In [10]:
offset_image = fourier_shift(np.fft.fftn(absolute2.data), -shift)
offset_image = np.fft.ifftn(offset_image)

In [15]:
vmin_mul = -10
vmax_mul = 10

fig, ax = plt.subplots(nrows = 2, ncols= 2)
image1 =absolute1.data

ax[0,0].set_title('holograph')
ax[0,0].imshow(image1, cmap='gray', \
                 vmin=vmin_mul*abs(np.mean(image1)), vmax=vmax_mul*abs(np.mean(image1)))

image2 =phase1.data
ax[0,1].set_title('holograph')
ax[0,1].imshow(image2, cmap='gray', \
                 vmin=vmin_mul*abs(np.mean(image2)), vmax=vmax_mul*abs(np.mean(image2)))

image3 =absolute2.data

ax[1,0].set_title('holograph')
ax[1,0].imshow(image1, cmap='gray', \
                 vmin=vmin_mul*abs(np.mean(image3)), vmax=vmax_mul*abs(np.mean(image3)))

image4 =phase2.data
ax[1,1].set_title('holograph')
ax[1,1].imshow(image2, cmap='gray', \
                 vmin=vmin_mul*abs(np.mean(image4)), vmax=vmax_mul*abs(np.mean(image4)))

In [14]:
vmin_mul = -10
vmax_mul = 10

fig, ax = plt.subplots(nrows =1, ncols= 3)
image1 =absolute1.data
ax[0].set_title('holograph')
ax[0].imshow(image1, cmap='gray', \
                 vmin=vmin_mul*abs(np.mean(image1)), vmax=vmax_mul*abs(np.mean(image1)))

image2 =offset_image.real
ax[1].set_title('holograph')
ax[1].imshow(image2, cmap='gray', \
                 vmin=vmin_mul*abs(np.mean(image2)), vmax=vmax_mul*abs(np.mean(image2)))

image3 =absolute1.data - offset_image.real

ax[2].set_title('holograph')
ax[2].imshow(image1, cmap='gray', \
                 vmin=vmin_mul*abs(np.mean(image3)), vmax=vmax_mul*abs(np.mean(image3)))


In [ ]:
offset_image.real

In [ ]:
image = data.camera()
shift = (-22.4, 13.32)
# The shift corresponds to the pixel offset relative to the reference image
offset_image = fourier_shift(np.fft.fftn(image), shift)
offset_image = np.fft.ifftn(offset_image)
print(f'Known offset (y, x): {shift}')

In [ ]:
image.shape

In [ ]:
offset_image